<a href="https://colab.research.google.com/github/seunghyeon98/S-bert-/blob/main/%EC%A0%9C%EC%A3%BC%EB%8F%84_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sentence_transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=c39c8545139994ceb0e717ddf180508adde20067f5b40a6dff0c73265c1bc945
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from textblob import TextBlob

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go

import pickle
from pprint import pprint
from tabulate import tabulate

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# pd.set_option('display.max_rows', None)

In [ ]:
model_args ={
'sbert_klue' : 'snunlp/KR-SBERT-V40K-klueNLI-augSTS',
 'sbert_sts':'jhgan/ko-sbert-sts',
'sroberta':'jhgan/ko-sroberta-multitask',
'albert':'bongsoo/albert-small-kor-sbert-v1.1'
}


In [ ]:
model = SentenceTransformer(model_args['sroberta'])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd/content/drive/MyDrive/제주도


/content/drive/.shortcut-targets-by-id/1010apg6IkfM7jRKG_xSrqA1L88EGq037/제주도


In [ ]:
# df_filltered_04131146 <- 디저트류인 클러스터에 해당하는 데이터들 지운 데이터
df = pd.read_pickle('df_filltered_04131146.pickle')
df

,Unnamed: 0,source,target,source_type,target_type,value
0,0,김치전골(돼지고기),"김치, 배추 김치",food,ingred,1.000
1,1,김치전골(돼지고기),"돼지고기, 사태, 삶은것",food,ingred,0.500
2,2,김치전골(돼지고기),두부,food,ingred,0.500
3,3,김치전골(돼지고기),"당근, 뿌리, 데친것",food,ingred,0.333
4,4,김치전골(돼지고기),"시금치, 데친것",food,ingred,0.333
...,...,...,...,...,...,...
8685,8685,깻잎장아찌,들깻잎장아찌,food,ingred,1.000
8686,8686,마늘장아찌,마늘 장아찌,food,ingred,1.000
8687,8687,양파장아찌,양파 장아찌,food,ingred,1.000
8688,8688,무초절임(쌈무),"무 절임, 쌈무",food,ingred,1.000


In [ ]:
# Using groupby() and agg() to combine the 'target' values for each unique 'source' value
combined_df = df.groupby('source')['target'].agg(', '.join).reset_index()

combined_df['ingredient_word'] = combined_df.apply(lambda x : x['target'].split(','),axis=1)


def remove_things(x):
    return [item for item in x if '것' not in str(item)]

combined_df['remove_word'] = combined_df['ingredient_word'].apply(remove_things)

combined_df['sentence'] = combined_df.apply(lambda x : x['source'] + ', ' + ",".join(x['remove_word']),axis=1 )
combined_df

,source,target,ingredient_word,remove_word,sentence
0,가오리구이,"가오리, 나비가오리, 생것, 콩기름, 소금","[가오리, 나비가오리, 생것, 콩기름, 소금]","[가오리, 나비가오리, 콩기름, 소금]","가오리구이, 가오리, 나비가오리, 콩기름, 소금"
1,가오리찜,"가오리, 나비가오리, 생것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, 양파...","[가오리, 나비가오리, 생것, 무, 조선무, 생것, 미나리, 물미나리,...","[가오리, 나비가오리, 무, 조선무, 미나리, 물미나리, 양파, 콩나물...","가오리찜, 가오리, 나비가오리, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, ..."
2,가자미구이,"가자미, 생것, 소금, 콩기름","[가자미, 생것, 소금, 콩기름]","[가자미, 소금, 콩기름]","가자미구이, 가자미, 소금, 콩기름"
3,가자미매운탕(고추장),"가자미, 생것, 무, 조선무, 생것, 콩나물, 삶은것, 간장, 재래, 고추, 풋고추...","[가자미, 생것, 무, 조선무, 생것, 콩나물, 삶은것, 간장, 재래...","[가자미, 무, 조선무, 콩나물, 간장, 재래, 고추, 풋고추, 쑥갓...","가자미매운탕(고추장), 가자미, 무, 조선무, 콩나물, 간장, 재래, 고추, 풋고추..."
4,가자미조림,"가자미, 생것, 무, 조선무, 생것, 간장, 개량, 양조, 양파, 데친것, 파, 실...","[가자미, 생것, 무, 조선무, 생것, 간장, 개량, 양조, 양파, ...","[가자미, 무, 조선무, 간장, 개량, 양조, 양파, 파, 실파, ...","가자미조림, 가자미, 무, 조선무, 간장, 개량, 양조, 양파, 파, 실파, 마늘,..."
...,...,...,...,...,...
1151,황새치튀김,"조기(참조기), 생것, 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금","[조기(참조기), 생것, 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기...","[조기(참조기), 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금]","황새치튀김, 조기(참조기), 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금"
1152,회냉면(홍어),"메밀 냉면, 말린것, 삶은것, 메밀 냉면 육수, 인스턴트, 홍어, 생것, 무 절임,...","[메밀 냉면, 말린것, 삶은것, 메밀 냉면 육수, 인스턴트, 홍어, 생것...","[메밀 냉면, 메밀 냉면 육수, 인스턴트, 홍어, 무 절임, 치킨무, 달...","회냉면(홍어), 메밀 냉면, 메밀 냉면 육수, 인스턴트, 홍어, 무 절임, 치킨무,..."
1153,"회덮밥(참치, 고추장)","다랑어, 참다랑어, 냉동, 고추장, 개량, 당근, 뿌리, 데친것, 양배추, 생것, ...","[다랑어, 참다랑어, 냉동, 고추장, 개량, 당근, 뿌리, 데친것, ...","[다랑어, 참다랑어, 냉동, 고추장, 개량, 당근, 뿌리, 양배추, ...","회덮밥(참치, 고추장), 다랑어, 참다랑어, 냉동, 고추장, 개량, 당근, 뿌리, ..."
1154,"회덮밥(참치, 초고추장)","다랑어, 참다랑어, 냉동, 멥쌀, 백미, 밥, 고추장, 개량, 무, 조선무, 생것,...","[다랑어, 참다랑어, 냉동, 멥쌀, 백미, 밥, 고추장, 개량, 무,...","[다랑어, 참다랑어, 냉동, 멥쌀, 백미, 밥, 고추장, 개량, 무,...","회덮밥(참치, 초고추장), 다랑어, 참다랑어, 냉동, 멥쌀, 백미, 밥, 고추장, ..."


In [ ]:
combined_df[combined_df['source'].str.contains('찜')]

,source,target,ingredient_word,remove_word,sentence
1,가오리찜,"가오리, 나비가오리, 생것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, 양파...","[가오리, 나비가오리, 생것, 무, 조선무, 생것, 미나리, 물미나리,...","[가오리, 나비가오리, 무, 조선무, 미나리, 물미나리, 양파, 콩나물...","가오리찜, 가오리, 나비가오리, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, ..."
5,가자미찜,"가자미, 생것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, 양파, 데친것, ...","[가자미, 생것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, ...","[가자미, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, 간장, ...","가자미찜, 가자미, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, 간장, 개량,..."
14,가지찜,"가지, 데친것, 소고기, 한우, 양지, 삶은것, 간장, 개량, 양조, 고추, 빨간색...","[가지, 데친것, 소고기, 한우, 양지, 삶은것, 간장, 개량, 양조...","[가지, 소고기, 한우, 양지, 간장, 개량, 양조, 고추, 빨간색,...","가지찜, 가지, 소고기, 한우, 양지, 간장, 개량, 양조, 고추, 빨간색, 밀, ..."
19,갈비찜,"소고기, 한우, 갈비, 생것, 무, 조선무, 생것, 당근, 뿌리, 데친것, 표고버섯...","[소고기, 한우, 갈비, 생것, 무, 조선무, 생것, 당근, 뿌리, ...","[소고기, 한우, 갈비, 무, 조선무, 당근, 뿌리, 표고버섯, 참나...","갈비찜, 소고기, 한우, 갈비, 무, 조선무, 당근, 뿌리, 표고버섯, 참나무재배,..."
25,갈치찜,"갈치, 생것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, 양파, 데친것, 콩...","[갈치, 생것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, 양...","[갈치, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, 간장, 개...","갈치찜, 갈치, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, 간장, 개량, 양..."
80,계란찜,"달걀, 삶은것, 양파, 데친것, 파, 대파, 생것, 소금","[달걀, 삶은것, 양파, 데친것, 파, 대파, 생것, 소금]","[달걀, 양파, 파, 대파, 소금]","계란찜, 달걀, 양파, 파, 대파, 소금"
100,고등어찜,"고등어, 삶은것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, 양파, 데친것,...","[고등어, 삶은것, 무, 조선무, 생것, 미나리, 물미나리, 데친것, ...","[고등어, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, 간장, ...","고등어찜, 고등어, 무, 조선무, 미나리, 물미나리, 양파, 콩나물, 간장, 개량,..."
101,고등어찜(자반고등어),"고등어, 자반고등어, 무, 조선무, 생것, 양파, 데친것, 고추, 풋고추, 생것, ...","[고등어, 자반고등어, 무, 조선무, 생것, 양파, 데친것, 고추, ...","[고등어, 자반고등어, 무, 조선무, 양파, 고추, 풋고추, 쑥갓, ...","고등어찜(자반고등어), 고등어, 자반고등어, 무, 조선무, 양파, 고추, 풋고추, ..."
192,깻잎찜,"들깻잎, 찐것, 고춧가루, 간장, 개량, 양조, 고추, 빨간색, 생것, 마늘, 생것...","[들깻잎, 찐것, 고춧가루, 간장, 개량, 양조, 고추, 빨간색, 생...","[들깻잎, 고춧가루, 간장, 개량, 양조, 고추, 빨간색, 마늘, 참...","깻잎찜, 들깻잎, 고춧가루, 간장, 개량, 양조, 고추, 빨간색, 마늘, 참기름, ..."
196,꼬막찜,"꼬막, 생것, 파, 쪽파, 생것, 간장, 개량, 양조, 고춧가루, 마늘, 생것, 설...","[꼬막, 생것, 파, 쪽파, 생것, 간장, 개량, 양조, 고춧가루, ...","[꼬막, 파, 쪽파, 간장, 개량, 양조, 고춧가루, 마늘, 설탕, ...","꼬막찜, 꼬막, 파, 쪽파, 간장, 개량, 양조, 고춧가루, 마늘, 설탕, 백설탕,..."


In [ ]:
encoded_data = model.encode(combined_df.sentence.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
# encoded_data

encoded_data_df =pd.DataFrame(encoded_data)
# encoded_data_df

In [ ]:
args ={
    'n_cluster' :8,
    'random_state' : 98
}

In [ ]:
kmeans = KMeans(n_clusters=args['n_cluster'],random_state=args['random_state'])
kmeans.fit(encoded_data)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

DF_labels = pd.DataFrame(labels)
# DF_labels
combined_df['Kmeans_clusters'] = DF_labels
# combined_df

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [ ]:
combined_df[combined_df['Kmeans_clusters']==7].sample(20)

,source,target,ingredient_word,remove_word,sentence,Kmeans_clusters
109,고추장,"고추장, 개량","[고추장, 개량]","[고추장, 개량]","고추장, 고추장, 개량",7
167,"김치찌개(돼지고기, 된장)","김치, 배추 김치, 두부, 돼지고기, 사태, 삶은것, 호박, 애호박, 데친것, 된장...","[김치, 배추 김치, 두부, 돼지고기, 사태, 삶은것, 호박, 애호박,...","[김치, 배추 김치, 두부, 돼지고기, 사태, 호박, 애호박, 된장, ...","김치찌개(돼지고기, 된장), 김치, 배추 김치, 두부, 돼지고기, 사태, 호박, 애...",7
314,"동태찌개(고추장, 간장)","명태, 구운것, 두부, 무, 조선무, 생것, 고추장, 개량, 마늘, 생것, 쑥갓, ...","[명태, 구운것, 두부, 무, 조선무, 생것, 고추장, 개량, 마늘,...","[명태, 두부, 무, 조선무, 고추장, 개량, 마늘, 쑥갓, 간장, ...","동태찌개(고추장, 간장), 명태, 두부, 무, 조선무, 고추장, 개량, 마늘, 쑥갓...",7
819,열무김치,"김치, 열무 김치","[김치, 열무 김치]","[김치, 열무 김치]","열무김치, 김치, 열무 김치",7
911,잉어찌개(고추장),"잉어, 생것, 무, 조선무, 생것, 두부, 고추장, 개량, 간장, 재래, 쑥갓, 데...","[잉어, 생것, 무, 조선무, 생것, 두부, 고추장, 개량, 간장, ...","[잉어, 무, 조선무, 두부, 고추장, 개량, 간장, 재래, 쑥갓, ...","잉어찌개(고추장), 잉어, 무, 조선무, 두부, 고추장, 개량, 간장, 재래, 쑥갓...",7
166,김치찌개(돼지고기),"김치, 배추 김치, 돼지고기, 사태, 삶은것, 두부, 양파, 데친것, 마늘, 생것,...","[김치, 배추 김치, 돼지고기, 사태, 삶은것, 두부, 양파, 데친것,...","[김치, 배추 김치, 돼지고기, 사태, 두부, 양파, 마늘, 고춧가루,...","김치찌개(돼지고기), 김치, 배추 김치, 돼지고기, 사태, 두부, 양파, 마늘, 고...",7
367,두부찌개,"두부, 무, 조선무, 생것, 파, 대파, 생것, 멸치 육수, 고춧가루, 마늘, 생것...","[두부, 무, 조선무, 생것, 파, 대파, 생것, 멸치 육수, 고춧가...","[두부, 무, 조선무, 파, 대파, 멸치 육수, 고춧가루, 마늘, 소금]","두부찌개, 두부, 무, 조선무, 파, 대파, 멸치 육수, 고춧가루, 마늘, 소금",7
369,"두부찌개(고추장, 된장)","두부, 된장, 개량, 고추장, 개량, 파, 대파, 생것, 멸치 육수, 고춧가루, 마...","[두부, 된장, 개량, 고추장, 개량, 파, 대파, 생것, 멸치 육수...","[두부, 된장, 개량, 고추장, 개량, 파, 대파, 멸치 육수, 고춧...","두부찌개(고추장, 된장), 두부, 된장, 개량, 고추장, 개량, 파, 대파, 멸치 ...",7
1031,코다리찌개(고추장),"명태, 코다리, 반건조, 구운것, 무, 조선무, 생것, 두부, 고추장, 개량, 간장...","[명태, 코다리, 반건조, 구운것, 무, 조선무, 생것, 두부, 고추...","[명태, 코다리, 반건조, 무, 조선무, 두부, 고추장, 개량, 간장...","코다리찌개(고추장), 명태, 코다리, 반건조, 무, 조선무, 두부, 고추장, 개량,...",7
1100,해물찌개(된장),"장어, 뱀장어, 생것, 넙치(광어), 생것, 무, 조선무, 생것, 느타리버섯, 데친...","[장어, 뱀장어, 생것, 넙치(광어), 생것, 무, 조선무, 생것, ...","[장어, 뱀장어, 넙치(광어), 무, 조선무, 느타리버섯, 미나리, 물...","해물찌개(된장), 장어, 뱀장어, 넙치(광어), 무, 조선무, 느타리버섯, 미나리,...",7


In [ ]:
for index, row in combined_df.iterrows():
    if row['Kmeans_clusters'] == 0:
        combined_df.loc[index, 'cluster_name'] = '반찬'
    elif row['Kmeans_clusters'] == 1:
        combined_df.loc[index, 'cluster_name'] = '전,튀김'
    elif row['Kmeans_clusters'] == 2:
        combined_df.loc[index, 'cluster_name'] = '메인요리'
    elif row['Kmeans_clusters'] == 3:
        combined_df.loc[index, 'cluster_name'] = '해산물요리'
    elif row['Kmeans_clusters'] == 4:
        combined_df.loc[index, 'cluster_name'] = '국'
    elif row['Kmeans_clusters'] == 5:
        combined_df.loc[index, 'cluster_name'] = '곡류'
    elif row['Kmeans_clusters'] == 6:
        combined_df.loc[index, 'cluster_name'] = '조림,찜'
    elif row['Kmeans_clusters'] == 7:
        combined_df.loc[index, 'cluster_name'] = '찌개요리'
    # elif row['Kmeans_clusters'] == 8:
    #     combined_df.loc[index, 'cluster_name'] = '튀김요리'
    # elif row['Kmeans_clusters'] == 9:
    #     combined_df.loc[index, 'cluster_name'] = '국요리'


In [ ]:
combined_df[combined_df['Kmeans_clusters']==1].sample(20)

,source,target,ingredient_word,remove_word,sentence,Kmeans_clusters,cluster_name,not_inter_per,recommend
12,가지전,"가지, 데친것, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금","[가지, 데친것, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금]","[가지, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금]","가지전, 가지, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금",1,"전,튀김",0.90,o
1089,피망전,"밀, 중력밀가루, 달걀, 부침(달걀프라이), 들깻잎, 데친것, 피망, 초록색, 데친것, 고추, 풋고추, 생것, 콩기름, 소금","[밀, 중력밀가루, 달걀, 부침(달걀프라이), 들깻잎, 데친것, 피망, 초록색, 데친것, 고추, 풋고추, 생것, 콩기름, 소금]","[밀, 중력밀가루, 달걀, 부침(달걀프라이), 들깻잎, 피망, 초록색, 고추, 풋고추, 콩기름, 소금]","피망전, 밀, 중력밀가루, 달걀, 부침(달걀프라이), 들깻잎, 피망, 초록색, 고추, 풋고추, 콩기름, 소금",1,"전,튀김",0.90,o
813,연근전,"연근, 생것, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금","[연근, 생것, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금]","[연근, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금]","연근전, 연근, 달걀, 부침(달걀프라이), 밀, 중력밀가루, 콩기름, 소금",1,"전,튀김",0.90,o
815,연근튀김,"연근, 생것, 달걀, 부침(달걀프라이), 밀, 튀김가루, 콩기름, 소금","[연근, 생것, 달걀, 부침(달걀프라이), 밀, 튀김가루, 콩기름, 소금]","[연근, 달걀, 부침(달걀프라이), 밀, 튀김가루, 콩기름, 소금]","연근튀김, 연근, 달걀, 부침(달걀프라이), 밀, 튀김가루, 콩기름, 소금",1,"전,튀김",0.90,o
990,참치튀김,"다랑어, 참다랑어, 구운것, 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금","[다랑어, 참다랑어, 구운것, 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금]","[다랑어, 참다랑어, 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금]","참치튀김, 다랑어, 참다랑어, 밀, 튀김가루, 달걀, 부침(달걀프라이), 콩기름, 소금",1,"전,튀김",0.90,o
68,계란(삶은것),"달걀, 삶은것","[달걀, 삶은것]",[달걀],"계란(삶은것), 달걀",1,"전,튀김",1.00,o
545,베이컨튀김,"베이컨, 달걀, 부침(달걀프라이), 전분, 감자, 가루, 콩기름, 소금, 후추, 검은색, 가루","[베이컨, 달걀, 부침(달걀프라이), 전분, 감자, 가루, 콩기름, 소금, 후추, 검은색, 가루]","[베이컨, 달걀, 부침(달걀프라이), 전분, 감자, 가루, 콩기름, 소금, 후추, 검은색, 가루]","베이컨튀김, 베이컨, 달걀, 부침(달걀프라이), 전분, 감자, 가루, 콩기름, 소금, 후추, 검은색, 가루",1,"전,튀김",0.85,o
76,"계란부침(양파, 당근)","달걀, 부침(달걀프라이), 당근, 뿌리, 데친것, 양파, 데친것, 파, 대파, 생것, 소금, 콩기름","[달걀, 부침(달걀프라이), 당근, 뿌리, 데친것, 양파, 데친것, 파, 대파, 생것, 소금, 콩기름]","[달걀, 부침(달걀프라이), 당근, 뿌리, 양파, 파, 대파, 소금, 콩기름]","계란부침(양파, 당근), 달걀, 부침(달걀프라이), 당근, 뿌리, 양파, 파, 대파, 소금, 콩기름",1,"전,튀김",0.95,o
1073,파전,"파, 쪽파, 생것, 양파, 데친것, 고추, 빨간색, 생것, 밀, 중력밀가루, 물, 증류수, 달걀, 부침(달걀프라이), 소금, 천일염, 가는소금, 콩기름","[파, 쪽파, 생것, 양파, 데친것, 고추, 빨간색, 생것, 밀, 중력밀가루, 물, 증류수, 달걀, 부침(달걀프라이), 소금, 천일염, 가는소금, 콩기름]","[파, 쪽파, 양파, 고추, 빨간색, 밀, 중력밀가루, 물, 증류수, 달걀, 부침(달걀프라이), 소금, 천일염, 가는소금, 콩기름]","파전, 파, 쪽파, 양파, 고추, 빨간색, 밀, 중력밀가루, 물, 증류수, 달걀, 부침(달걀프라이), 소금, 천일염, 가는소금, 콩기름",1,"전,튀김",0.90,o
189,깻잎전,"밀, 중력밀가루, 들깻잎, 데친것, 달걀, 부침(달걀프라이), 콩기름, 소금","[밀, 중력밀가루, 들깻잎, 데친것, 달걀, 부침(달걀프라이), 콩기름, 소금]","[밀, 중력밀가루, 들깻잎, 달걀, 부침(달걀프라이), 콩기름, 소금]","깻잎전, 밀, 중력밀가루, 들깻잎, 달걀, 부침(달걀프라이), 콩기름, 소금",1,"전,튀김",0.90,o


In [ ]:
perplexity = 1
if perplexity >= len(combined_df):
    perplexity = len(combined_df) - 1

In [ ]:
tsne_np = TSNE(n_components=2,perplexity=perplexity,
               random_state=args['random_state']).fit_transform(encoded_data_df)
tsne_df = pd.DataFrame(tsne_np,columns=['comp_0','comp_1'])
tsne_df

,comp_0,comp_1
0,-40.656940,-82.369827
1,-41.232708,-81.905289
2,43.919506,-74.139130
3,-36.219501,-45.572021
4,-35.510689,-44.663445
...,...,...
1151,62.865940,-72.592293
1152,-50.497997,58.448669
1153,-29.127182,-16.420059
1154,-29.290438,-16.578485


In [ ]:
tsne_table = pd.concat([combined_df.loc[:,('source','Kmeans_clusters','cluster_name')] , tsne_df],axis=1)
# tsne_table

In [ ]:
fig = px.scatter(data_frame = tsne_table,
                 x='comp_0',y='comp_1',
                 color='cluster_name',
                 hover_data=['source'])

# fig.update_traces(marker=dict(size=10))
fig.update_traces(marker=dict(size=7, sizemode='area', sizemin=100))

fig.update_layout(
    width=600,
    height=600
)

fig.show()

# input

In [ ]:
# input_query ='회냉면(홍어)'
input_query ='파전'


In [ ]:
threshold = 0.7

main_threshold = 0.5


groups = combined_df.groupby(combined_df.Kmeans_clusters)



# 분모

query = sum(combined_df[combined_df['source']==input_query]['remove_word'].tolist(),[])
query_cluster = combined_df[combined_df['source']==input_query]['Kmeans_clusters'].iloc[0]

query_len = len(query)

# 클러스터 마다, 먹은 음식의 재료와 겹치지 않는 것을 뽑기
# 쿼리와의 교집합 값 계산하기

for cluster_type in tqdm(range(args['n_cluster'])):
  for idx in (groups.groups[cluster_type]):

    comparison_target = combined_df.loc[idx,'remove_word']
      
    intersection = set(query).intersection(comparison_target)

    # 분자
    intersection_len = len(intersection)

    # 안겹치는 정도 계산하기
    calculated = 1 - (intersection_len / query_len)

    combined_df.loc[idx,'not_inter_per'] = calculated
    
    if combined_df.loc[idx,'Kmeans_clusters'] == 2:
      if calculated > main_threshold:
        combined_df.loc[idx,'recommend'] ='o'
      else:
        combined_df.loc[idx,'recommend'] ='x'

    else: 
      # 안겹치는 정도가 특정 임계값 이상인것만
      if calculated > threshold:
        combined_df.loc[idx,'recommend'] ='o'
      else:
        combined_df.loc[idx,'recommend'] ='x'
    



100%|██████████| 8/8 [00:00<00:00, 13.37it/s]


# 클러스터별 통계

In [ ]:
grouped = combined_df.groupby(['Kmeans_clusters','recommend'])
counted = grouped.count()
counted.reset_index(inplace=True)
counted
over_threshold_count = counted[counted['recommend']=='o']['source'].tolist()

cluster_threshold_per = combined_df['Kmeans_clusters'].value_counts().sort_index().to_frame()
cluster_threshold_per
cluster_threshold_per['over_threshold_count'] = over_threshold_count
cluster_threshold_per

,Kmeans_clusters,over_threshold_count
0,149,147
1,140,72
2,181,178
3,60,60
4,200,152
5,75,65
6,221,212
7,130,128


In [ ]:
# 클러스터마다 데이터프레임을 cluster_dfs[0]~[n_cluster] 까지 저장하여, not_inter_per 의 순대로, 내림차순 정렬하기
cluster_in_Qvalue_dfs = []
for i in range(args['n_cluster']):
    cluster_name = 'cluster_'+str(i)
    if cluster_name in globals():
        cluster_df = globals()[cluster_name]
    else:
        cluster_df = pd.DataFrame()
    cluster_df = combined_df[combined_df['Kmeans_clusters']==i].sort_values('not_inter_per',ascending=False)
    
    Qvalue = cluster_df['not_inter_per'].quantile(0.25)
    
    cluster_in_Qvalue = cluster_df[(cluster_df['not_inter_per']<=Qvalue)&(cluster_df['recommend']=='o')]
    cluster_in_Qvalue = cluster_in_Qvalue.head(20)
    cluster_in_Qvalue_dfs.append(cluster_in_Qvalue)

In [ ]:
cluster_in_Qvalue_df = pd.concat(cluster_in_Qvalue_dfs)
# cluster_in_Qvalue_df

In [ ]:
# 클러스터별 추천할 수 있는 음식이 몇개있는지
cluster_in_Qvalue_df['Kmeans_clusters'].value_counts()

0    20
2    20
3    20
4    20
5    20
6    20
7    20
Name: Kmeans_clusters, dtype: int64

In [ ]:
# 쿼리문에 해당하지 않은 클러스터들 추출
recommend_table = cluster_in_Qvalue_df[cluster_in_Qvalue_df['Kmeans_clusters']!=query_cluster]
recommend_table.head()

,source,target,ingredient_word,remove_word,sentence,Kmeans_clusters,cluster_name,not_inter_per,recommend
297,도라지볶음(간장),"도라지, 뿌리, 데친것, 파, 실파, 생것, 콩기름, 간장, 재래, 마늘, 생것, 소금, 참기름, 참깨, 흰색, 볶은것","[도라지, 뿌리, 데친것, 파, 실파, 생것, 콩기름, 간장, 재래, 마늘, 생것, 소금, 참기름, 참깨, 흰색, 볶은것]","[도라지, 뿌리, 파, 실파, 콩기름, 간장, 재래, 마늘, 소금, 참기름, 참깨, 흰색]","도라지볶음(간장), 도라지, 뿌리, 파, 실파, 콩기름, 간장, 재래, 마늘, 소금, 참기름, 참깨, 흰색",0,반찬,0.866667,o
296,도라지볶음,"도라지, 뿌리, 데친것, 콩기름, 파, 대파, 생것, 깨소금, 가루, 볶은것, 마늘, 생것, 소금, 참기름","[도라지, 뿌리, 데친것, 콩기름, 파, 대파, 생것, 깨소금, 가루, 볶은것, 마늘, 생것, 소금, 참기름]","[도라지, 뿌리, 콩기름, 파, 대파, 깨소금, 가루, 마늘, 소금, 참기름]","도라지볶음, 도라지, 뿌리, 콩기름, 파, 대파, 깨소금, 가루, 마늘, 소금, 참기름",0,반찬,0.866667,o
294,도라지구이(고추장),"도라지, 뿌리, 데친것, 고추장, 개량, 콩기름, 파, 대파, 생것, 참기름, 마늘, 생것, 설탕, 백설탕, 소금","[도라지, 뿌리, 데친것, 고추장, 개량, 콩기름, 파, 대파, 생것, 참기름, 마늘, 생것, 설탕, 백설탕, 소금]","[도라지, 뿌리, 고추장, 개량, 콩기름, 파, 대파, 참기름, 마늘, 설탕, 백설탕, 소금]","도라지구이(고추장), 도라지, 뿌리, 고추장, 개량, 콩기름, 파, 대파, 참기름, 마늘, 설탕, 백설탕, 소금",0,반찬,0.866667,o
833,오이나물,"오이, 취청, 생것, 양파, 데친것, 마늘, 생것, 참깨, 흰색, 볶은것, 깨소금, 가루, 볶은것, 소금, 참기름","[오이, 취청, 생것, 양파, 데친것, 마늘, 생것, 참깨, 흰색, 볶은것, 깨소금, 가루, 볶은것, 소금, 참기름]","[오이, 취청, 양파, 마늘, 참깨, 흰색, 깨소금, 가루, 소금, 참기름]","오이나물, 오이, 취청, 양파, 마늘, 참깨, 흰색, 깨소금, 가루, 소금, 참기름",0,반찬,0.866667,o
837,오이볶음,"오이, 다다기, 생것, 파, 실파, 생것, 콩기름, 마늘, 생것, 참기름, 소금, 참깨, 흰색, 볶은것","[오이, 다다기, 생것, 파, 실파, 생것, 콩기름, 마늘, 생것, 참기름, 소금, 참깨, 흰색, 볶은것]","[오이, 다다기, 파, 실파, 콩기름, 마늘, 참기름, 소금, 참깨, 흰색]","오이볶음, 오이, 다다기, 파, 실파, 콩기름, 마늘, 참기름, 소금, 참깨, 흰색",0,반찬,0.866667,o


In [ ]:
recommend_clusters = []
for i in range(args['n_cluster']):
  cluster_name = 'cluster_' + str(i)

  if cluster_name in globals():
    cluster_df = globals()[cluster_name]
  else:
    cluster_df = pd.DataFrame()
  cluster_df = recommend_table[recommend_table['Kmeans_clusters']==i]
  recommend_clusters.append(cluster_df)

In [ ]:
# 각 클러스터별 일분위수에 해당하는 음식을 랜덤으로 5개 선별하는 작업
recommend_food = []
for i in range(args['n_cluster']):
  if recommend_clusters[i].empty:
    pass
  else:
    # recommend_food.append(recommend_clusters[i].sample(5))
    recommend_food.append(recommend_clusters[i].head(3))


In [ ]:
#@title 기본 제목 텍스트
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

print('입력한 음식 :',input_query)
print('입력한 음식의 클러스터:',query_cluster)

for i in range(args['n_cluster']):

  try:
    temp = recommend_food[i].drop(['remove_word','sentence','target','ingredient_word'],axis=1)
    print(temp)
    print('--------------------------------------------------------------------------------------------')
    # print(tabulate(temp,headers='keys',tablefmt='tsv',showindex=False))
  except:
    pass

입력한 음식 : 파전
입력한 음식의 클러스터: 1
         source  Kmeans_clusters cluster_name  not_inter_per recommend
297  도라지볶음(간장)   0                반찬           0.866667       o       
296  도라지볶음       0                반찬           0.866667       o       
294  도라지구이(고추장)  0                반찬           0.866667       o       
--------------------------------------------------------------------------------------------
    source  Kmeans_clusters cluster_name  not_inter_per recommend
138  궁중떡볶이  2                메인요리         0.733333       o       
251  닭개장    2                메인요리         0.733333       o       
903  육회비빔밥  2                메인요리         0.733333       o       
--------------------------------------------------------------------------------------------
    source  Kmeans_clusters cluster_name  not_inter_per recommend
866  옥돔구이   3                해산물요리        0.866667       o       
645  서대찜    3                해산물요리        0.866667       o       
936  전어구이   3                해산물요리      

<ipython-input-99-d532ec12b433>:5: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

